# Preparación de Datos para Predicción de Criptomonedas


Este notebook implementa la parte 3 de la consigna, utilizando el archivo CSV existente en lugar de volver a conectarse a la base de datos.

In [1]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import stats

# Configuración visual
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['figure.dpi'] = 100
warnings.filterwarnings('ignore')

# Configurar opciones de visualización para pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
# Instalamos el paquete holidays si no está instalado
%pip install holidays

In [2]:
# Importar holidays después de instalarlo
import holidays

## 1. Carga de Datos desde el CSV existente

En lugar de volver a conectarnos a la base de datos, utilizaremos el archivo CSV que ya hemos generado en el notebook anterior.

In [3]:
# Cargar datos desde el CSV existente
all_data = pd.read_csv('../data/processed_crypto_data.csv')

# Convertir la columna de fecha a datetime
all_data['fetch_date'] = pd.to_datetime(all_data['fetch_date'])

# Verificar datos cargados
print("Número de registros por moneda:")
print(all_data.groupby('coin_id').size())
print("\nRango de fechas disponibles:")
print(f"Desde: {all_data['fetch_date'].min()}")
print(f"Hasta: {all_data['fetch_date'].max()}")

# Mostrar las primeras filas para entender la estructura
all_data.head()

Número de registros por moneda:
coin_id
bitcoin     111
cardano     130
ethereum    123
dtype: int64

Rango de fechas disponibles:
Desde: 2025-01-01 00:00:00
Hasta: 2025-05-15 00:00:00


,coin_id,price_usd,fetch_date,pct_change,year,month,year_month,risk_level,rolling_7d_trend,rolling_7d_variance
0,bitcoin,93507.858747,2025-01-01,NaN,2025,1,2025-01,Low Risk,NaN,NaN
1,bitcoin,94384.176115,2025-01-02,0.937159,2025,1,2025-01,Low Risk,NaN,NaN
2,bitcoin,96852.146812,2025-01-03,2.614814,2025,1,2025-01,Low Risk,NaN,NaN
3,bitcoin,98084.342793,2025-01-04,1.272244,2025,1,2025-01,Low Risk,NaN,NaN
4,bitcoin,98256.738768,2025-01-05,0.175763,2025,1,2025-01,Low Risk,NaN,NaN


## 2. Reestructuración de los Datos para Predicción

Siguiendo la consigna, vamos a:
1. Agregar los precios de los últimos 7 días como columnas
2. Agregar el precio del día siguiente como variable objetivo
3. Agregar características adicionales para mejorar la predicción

In [4]:
# Proceso por cada moneda
coins = all_data['coin_id'].unique()
prediction_dfs = {}

for coin in coins:
    print(f"\nProcesando datos para {coin}...")
    
    # Filtrar por moneda y ordenar por fecha
    coin_data = all_data[all_data['coin_id'] == coin].sort_values('fetch_date').copy()
    
    # Asegurarnos que los datos estén ordenados cronológicamente
    coin_data = coin_data.sort_values('fetch_date').reset_index(drop=True)
    
    # a. Agregar características de precios de los últimos 7 días
    for i in range(1, 8):
        coin_data[f'price_lag_{i}'] = coin_data['price_usd'].shift(i)
    
    # b. Agregar el precio del día siguiente (target)
    coin_data['next_day_price'] = coin_data['price_usd'].shift(-1)
    
    # c. Feature engineering adicional - Asimetría (skewness) de los precios en ventana de 7 días
    coin_data['price_skew_7d'] = coin_data['price_usd'].rolling(window=7).skew()
    
    # d. Características basadas en el tiempo
    # Día de la semana (0=lunes, 6=domingo)
    coin_data['day_of_week'] = coin_data['fetch_date'].dt.dayofweek
    
    # Nombre del día de la semana
    day_names = {0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sábado', 6: 'Domingo'}
    coin_data['day_name'] = coin_data['day_of_week'].map(day_names)
    
    # Fin de semana (1=sí, 0=no)
    coin_data['is_weekend'] = coin_data['day_of_week'].isin([5, 6]).astype(int)
    
    # Semana del año
    coin_data['week_of_year'] = coin_data['fetch_date'].dt.isocalendar().week
    
    # Día del mes
    coin_data['day_of_month'] = coin_data['fetch_date'].dt.day
    
    # Cuarto de año (trimestre)
    coin_data['quarter'] = coin_data['fetch_date'].dt.quarter
    
    # Día del año
    coin_data['day_of_year'] = coin_data['fetch_date'].dt.dayofyear
    
    # e. Características de los días festivos (aunque no tenemos datos de volumen en el CSV existente)
    # Podemos agregar características ficticias usando el retorno diario como proxy de volumen
    coin_data['return_abs'] = coin_data['pct_change'].abs()
    coin_data['return_rolling_mean_7d'] = coin_data['return_abs'].rolling(window=7).mean()
    
    # f. Características de los días festivos
    # Festivos de EE.UU.
    us_holidays = holidays.US()
    coin_data['is_us_holiday'] = coin_data['fetch_date'].apply(lambda x: x in us_holidays).astype(int)
    
    # Festivos de China
    cn_holidays = holidays.China()
    coin_data['is_china_holiday'] = coin_data['fetch_date'].apply(lambda x: x in cn_holidays).astype(int)
    
    # Día previo a festivo (EE.UU.)
    coin_data['is_pre_us_holiday'] = coin_data['fetch_date'].apply(
        lambda x: (x + timedelta(days=1)) in us_holidays).astype(int)
    
    # g. Escalado de características
    # Crear una versión normalizada del precio (0-1)
    scaler = MinMaxScaler()
    coin_data['price_normalized'] = scaler.fit_transform(coin_data[['price_usd']])
    
    # Standardización del precio (media=0, std=1)
    coin_data['price_standardized'] = stats.zscore(coin_data['price_usd'], nan_policy='omit')
    
    # Eliminar filas con valores NaN (causados por los lags)
    coin_data = coin_data.dropna()
    
    # Guardar dataframe procesado
    prediction_dfs[coin] = coin_data

# Mostrar un ejemplo del dataframe resultante
print("\nEjemplo de dataframe procesado para Bitcoin:")
display(prediction_dfs['bitcoin'].head())


Procesando datos para bitcoin...

Procesando datos para cardano...

Procesando datos para ethereum...

Ejemplo de dataframe procesado para Bitcoin:


,coin_id,price_usd,fetch_date,pct_change,year,month,year_month,risk_level,rolling_7d_trend,rolling_7d_variance,price_lag_1,price_lag_2,price_lag_3,price_lag_4,price_lag_5,price_lag_6,price_lag_7,next_day_price,price_skew_7d,day_of_week,day_name,is_weekend,week_of_year,day_of_month,quarter,day_of_year,return_abs,return_rolling_mean_7d,is_us_holiday,is_china_holiday,is_pre_us_holiday,price_normalized,price_standardized
7,bitcoin,96952.098868,2025-01-08,-5.162210,2025,1,2025-01,Low Risk,3.683370,6.270013e+06,102229.394532,98364.589466,98256.738768,98084.342793,96852.146812,94384.176115,93507.858747,95016.714410,0.679446,2,Miércoles,0,2,8,1,8,5.162210,2.028717,0,0,0,0.690815,0.387952
8,bitcoin,95016.714410,2025-01-09,-1.996227,2025,1,2025-01,Low Risk,0.670174,5.077664e+06,96952.098868,102229.394532,98364.589466,98256.738768,98084.342793,96852.146812,94384.176115,92376.275783,1.066535,3,Jueves,0,2,9,1,9,1.996227,2.180012,0,0,0,0.625985,0.130462
9,bitcoin,92376.275783,2025-01-10,-2.778920,2025,1,2025-01,Low Risk,-4.621344,7.091491e+06,95016.714410,96952.098868,102229.394532,98364.589466,98256.738768,98084.342793,96852.146812,94736.626742,-0.107561,4,Viernes,0,2,10,1,10,2.778920,2.203456,0,0,0,0.537537,-0.220831
10,bitcoin,94736.626742,2025-01-11,2.555148,2025,1,2025-01,Low Risk,-3.413099,7.800154e+06,92376.275783,95016.714410,96952.098868,102229.394532,98364.589466,98256.738768,98084.342793,94559.551672,0.411438,5,Sábado,1,2,11,1,11,2.555148,2.386728,0,0,0,0.616603,0.093198
11,bitcoin,94559.551672,2025-01-12,-0.186913,2025,1,2025-01,Low Risk,-3.762782,8.205373e+06,94736.626742,92376.275783,95016.714410,96952.098868,102229.394532,98364.589466,98256.738768,94454.770896,0.991064,6,Domingo,1,2,12,1,12,0.186913,2.388321,0,0,0,0.610671,0.069639


In [5]:
# Guardar los datos procesados para cada moneda
for coin, df in prediction_dfs.items():
    output_path = f"../data/{coin}_prediction_data.csv"
    df.to_csv(output_path, index=False)
    print(f"Datos de {coin} guardados en {output_path}")

Datos de bitcoin guardados en ../data/bitcoin_prediction_data.csv
Datos de cardano guardados en ../data/cardano_prediction_data.csv
Datos de ethereum guardados en ../data/ethereum_prediction_data.csv


## 3. Visualización de características

Veamos cómo se relacionan las distintas características con el precio del día siguiente.

In [6]:
# Análisis de correlación para Bitcoin
btc_data = prediction_dfs['bitcoin']

# Calcular correlaciones con el precio del día siguiente
correlation = btc_data.corr()['next_day_price'].sort_values(ascending=False)
correlation = correlation.drop('next_day_price')  # Quitar autocorrelación
top_corr = correlation.head(15)

# Visualizar las correlaciones
plt.figure(figsize=(14, 10))
sns.barplot(x=top_corr.values, y=top_corr.index)
plt.title('Top 15 características con mayor correlación con el precio de mañana (Bitcoin)')
plt.xlabel('Correlación')
plt.tight_layout()
plt.savefig('../data/btc_top_correlations.png')
plt.show()

ValueError: could not convert string to float: 'bitcoin'

In [ ]:
# Visualización del precio por día de la semana
plt.figure(figsize=(14, 7))
sns.boxplot(x='day_name', y='price_usd', data=btc_data, 
            order=['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo'])
plt.title('Distribución de precios de Bitcoin por día de la semana')
plt.xlabel('Día de la semana')
plt.ylabel('Precio (USD)')
plt.xticks(rotation=45)
plt.tight_layout()
#plt.savefig('../data/btc_price_by_weekday.png')
plt.show()

In [ ]:
# Análisis de la influencia de los días festivos
plt.figure(figsize=(14, 7))
sns.boxplot(x='is_us_holiday', y='pct_change', data=btc_data)
plt.title('Cambio porcentual en precio de Bitcoin en días festivos vs. no festivos (EE.UU.)')
plt.xticks([0, 1], ['No festivo', 'Festivo'])
plt.xlabel('Día festivo en EE.UU.')
plt.ylabel('Cambio porcentual (%)')
plt.tight_layout()
plt.savefig('../data/btc_pct_change_by_holiday.png')
plt.show()

## Conclusión

Hemos reestructurado con éxito los datos del CSV existente para adaptarlos a la tarea de predicción. Ahora tenemos:

1. Los precios de los últimos 7 días como características
2. El precio del día siguiente como variable objetivo
3. Características adicionales relacionadas con el tiempo y días festivos
4. Versiones normalizadas y estandarizadas de los precios

Estos datos están listos para ser utilizados en un modelo de predicción de series temporales.